In [34]:
# this cell enables relative path imports
import os
from dotenv import load_dotenv
load_dotenv()
_PROJECT_PATH: str = os.environ["_project_path"]
_PICKLED_DATA_FILENAME: str = os.environ["_pickled_data_filename"]

import sys
from pathlib import Path
project_path = Path(_PROJECT_PATH)
sys.path.append(str(project_path))

In [35]:
# import all your modules here
import json
import pandas as pd

import config_v2 as cfg
from library_ubidots_v2 import Ubidots as ubi

In [3]:
# set your constants here
baseline=cfg.BASELINE
study=cfg.STUDY

In [4]:
df_devices = ubi.get_available_devices_v2(label='bancolombia', level='group', page_size=1000)
df_vars = ubi.get_available_variables(list(df_devices['device_id']))

In [5]:
df_vars = df_vars[df_vars['variable_label'].isin(cfg.WHITELISTED_VAR_LABELS)]
VAR_IDS_TO_REQUEST = list(df_vars['variable_id'])
VAR_ID_TO_LABEL = dict(zip(df_vars['variable_id'], df_vars['variable_label']))

In [6]:
SUBSET_SIZE = 45
DATE_INTERVALS_REQUEST = [
    [baseline[0], baseline[1]],
    [study[0], study[1]]
]

df = ubi.get_data_robust(VAR_IDS_TO_REQUEST, DATE_INTERVALS_REQUEST, SUBSET_SIZE)

Progress: 15.3%
Progress: 30.3%
Progress: 45.3%
Progress: 60.3%
Progress: 75.3%
Progress: 90.3%
Progress: 100.0%


In [36]:
pd.to_pickle(df, project_path / 'data'/ _PICKLED_DATA_FILENAME)

NameError: name 'df' is not defined

In [39]:
excel_mensual = pd.read_pickle(project_path / 'data'/ _PICKLED_DATA_FILENAME)
excel_mensual

,value,variable_id,variable_label,device_label,device_name,timestamp
datetime,,,,,,
2022-10-31 00:00:00-05:00,4.21,636979d877f8ee519b13817d,consumo-energia-reactiva-total,bc-816-1---calle-10,BC 816 - Calle 10,1667192400000
2022-10-30 23:00:00-05:00,4.17,636979d877f8ee519b13817d,consumo-energia-reactiva-total,bc-816-1---calle-10,BC 816 - Calle 10,1667188800000
2022-10-30 22:00:00-05:00,4.24,636979d877f8ee519b13817d,consumo-energia-reactiva-total,bc-816-1---calle-10,BC 816 - Calle 10,1667185200000
2022-10-30 21:00:00-05:00,4.27,636979d877f8ee519b13817d,consumo-energia-reactiva-total,bc-816-1---calle-10,BC 816 - Calle 10,1667181600000
2022-10-30 20:00:00-05:00,4.26,636979d877f8ee519b13817d,consumo-energia-reactiva-total,bc-816-1---calle-10,BC 816 - Calle 10,1667178000000
...,...,...,...,...,...,...
2022-11-01 00:14:47-05:00,8.19,61b7509cb5dc4f30080aed8b,front-potencia-activa,bc01-iluminacion-girardot,BC 659 - Girardot,1667279687000
2022-11-01 00:11:47-05:00,8.19,61b7509cb5dc4f30080aed8b,front-potencia-activa,bc01-iluminacion-girardot,BC 659 - Girardot,1667279507000
2022-11-01 00:08:47-05:00,8.17,61b7509cb5dc4f30080aed8b,front-potencia-activa,bc01-iluminacion-girardot,BC 659 - Girardot,1667279327000


In [40]:
excel_mensual = pd.read_pickle(project_path / 'data'/ _PICKLED_DATA_FILENAME)
excel_mensual = excel_mensual.groupby(by=["datetime", "variable_label", "device_name"]).resample('h').sum()
excel_mensual.drop(columns=['timestamp'], inplace=True)


In [31]:
excel_mensual.to_csv(project_path / 'data'/ 'excel_mensual.csv')

In [9]:
# df_coords = Report.get_coords_dataframe(list(df_devices['id']))
# pd.to_pickle(df_coords, "data_coordinates.pkl")

In [10]:
# df = pd.read_pickle(project_path / 'data'/ _PICKLED_DATA_FILENAME)

# df_cc_unico = df.query("device_name == 'BC 514 - Centro Comercial Único'")

# df_cc_unico_pa = df_cc_unico.query("variable_label == 'front-potencia-activa'").copy()
# df_cc_unico_cons = df_cc_unico.query("variable_label == 'front-consumo-activa'").copy()

# df_cc_unico_pa.value.plot()

In [11]:
# df_cc_unico_cons.value.plot()